# Cross Reference PDBBind and Binding DB

In [1]:
from rdkit import Chem
import pandas as pd
from pathlib import Path

In [2]:
pdbind_path = "/home/brennera/mystore/data/pdbbind_cn/data/"

In [3]:
df = pd.read_parquet(pdbind_path+'shortcut_direct/pdbbind.parquet')

In [4]:
df

,name,seq,smiles,receptor_xyz,ligand_xyz,ligand_rot,receptor_rot
0,10gs,PYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKASC...,[NH3+]C(CCC(=O)NC(CSCc1ccccc1)C(=O)NC(C(=O)O)c...,"[[31.19499969482422, 2.3919999599456787, 37.96...","[[15.088, 10.798, 23.547], [15.01, 9.987, 24.7...","[[-0.05242307217333261, 0.7927336266608463, 0....","[[0.0316964072031235, -0.8262117669010252, -0...."
1,184l,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,CC(C)Cc1ccccc1,"[[43.275001525878906, -1.8930000066757202, 9.1...","[[27.868, 8.799, 0.291], [27.165, 8.338, 1.533...","[[-0.4687402125675136, -0.17548961799160206, -...","[[0.3719438219898573, -0.8506274689677508, -0...."
2,186l,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,CCCCc1ccccc1,"[[43.44300079345703, -1.934999942779541, 9.173...","[[27.533, 9.085, 0.602], [26.645, 9.142, 1.809...","[[-0.5921787333551479, 0.799704214798722, -0.0...","[[0.33945760074297704, -0.8801912178972335, -0..."
3,187l,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,Cc1ccc(C)cc1,"[[43.487998962402344, -1.9119999408721924, 9.1...","[[26.137, 5.226, 5.77], [26.645, 6.141, 4.668]...","[[0.33426167353815706, -0.0708769321266389, -0...","[[0.3705288032053424, -0.8696608097391245, -0...."
4,188l,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,Cc1ccccc1C,"[[43.42499923706055, -1.9529999494552612, 9.06...","[[27.643, 8.878, 4.796], [27.214, 7.465, 4.489...","[[-0.2844330448678721, -0.4581528424684947, -0...","[[0.4373747982419609, -0.8288404178881474, -0...."
...,...,...,...,...,...,...,...
16074,8lpr,ANIVGGIEYSINNASLCSVGFSVTRGATKGFVTAGHCGTVNATARI...,CC([NH3+])C(=O)NC(C)C(=O)N1CCCC1C(=O)NC(Cc1ccc...,"[[22.618999481201172, 11.697999954223633, 6.42...","[[15.318, 40.262, 19.53], [14.188, 40.577, 18....","[[-0.7315256739317011, -0.1608592707380658, 0....","[[0.01076289065287429, 0.9505829275866897, -0...."
16075,9abp,NLKLGFLVKQPEEPWFQTEWKFADKAGKDLGFEVIKIAVPDGEKTL...,OCC1OC(O)C(O)C(O)C1O,"[[19.433000564575195, 26.624000549316406, 53.5...","[[17.026, 55.56, 51.771], [15.762, 56.156, 51....","[[-0.8818114160184656, -0.3756709342433778, 0....","[[0.07957388402083165, 0.17021830523645542, 0...."
16076,9hvp,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,CC(C)C(NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccccc1)C(O)...,"[[-12.961000442504883, 39.058998107910156, 30....","[[-8.076, 17.032, 32.146], [-8.749, 15.738, 32...","[[-0.44291261627667133, 0.20117857981284645, -...","[[-0.42918190818051605, -0.7868976344043886, 0..."
16077,9icd,SKVVVPAQGKKITLQNGKLNVPENPIIPYIEGDGIGVDVTPAMLKV...,Nc1ncnc2c1ncn2C1OC(COP(=O)(O)O)C(O)C1OP(=O)(O)O,"[[95.06400299072266, 41.05799865722656, 11.272...","[[98.333, 69.74, 1.429], [98.285, 70.597, 0.44...","[[-0.03673899529635519, -0.769273688619604, -0...","[[-0.24225787588618167, 0.7594609998636159, -0..."


In [5]:
pdbbind_proteins = df['name'].unique()

In [5]:
filename = "/home/brennera/mystore/data/bind_db/full_bindingdb/BindingDB_All_3D.sdf"  # replace with actual path
supplier = Chem.ForwardSDMolSupplier(filename)

unique_pdb_ids = set()
batch_size = 5000
batch = []

for i, mol in enumerate(supplier):
    if mol is None:
        continue

    batch.append({'ROMol': mol, **mol.GetPropsAsDict()})

    if len(batch) == batch_size:
        df = pd.DataFrame(batch)

        pdb_id_lists = [
            [pdb.strip() for pdb in str(pdb_string).split(',') if pdb.strip()]
            if pd.notna(pdb_string) else []
            for pdb_string in df['PDB ID(s) of Target Chain']
        ]

        unique_pdb_ids.update(pdb_id for sublist in pdb_id_lists for pdb_id in sublist)

        batch = []

# Process remaining molecules
if batch:
    df = pd.DataFrame(batch)
    pdb_id_lists = [
        [pdb.strip() for pdb in str(pdb_string).split(',') if pdb.strip()]
        if pd.notna(pdb_string) else []
        for pdb_string in df['PDB ID(s) of Target Chain']
    ]
    unique_pdb_ids.update(pdb_id for sublist in pdb_id_lists for pdb_id in sublist)

# Final flat, sorted list of unique PDB IDs
flat_unique_pdb_ids = sorted(unique_pdb_ids)


OSError: File error: Bad input file /home/brennera/mystore/data/bind_db/full_bindin

In [ ]:
flat_unique_pdb_ids